In [1]:
import pandas as pd

In [2]:
GS_data_set = {}
CS_data_set = {}

for year in range(2013,2026):
    GS_data_set[year] = pd.read_parquet(f"C:/Users/31155/Dropbox/EV-GasDualNetwork/Data/intermediate/yiwei/intermidiate/GS_cleaned/GS_data_cleaned_{year}.parquet")
    print(year," GS ",len(GS_data_set[year]))
for year in range(2015,2026):
    CS_data_set[year] = pd.read_parquet(f"C:/Users/31155/Dropbox/EV-GasDualNetwork/Data/intermediate/yiwei/intermidiate/CS_cleaned/CS_data_cleaned_{year}.parquet")
    print(year," CS ",len(CS_data_set[year]))

2013  GS  98458
2014  GS  102642
2015  GS  117307
2016  GS  118338
2017  GS  118193
2018  GS  104177
2019  GS  109464
2020  GS  113399
2021  GS  116279
2022  GS  105157
2023  GS  112000
2024  GS  113219
2025  GS  114421
2015  CS  1829
2016  CS  1870
2017  CS  4220
2018  CS  33175
2019  CS  56250
2020  CS  72149
2021  CS  93798
2022  CS  96181
2023  CS  118293
2024  CS  182254
2025  CS  265110


In [3]:
import numpy as np

match_columns = ['pname','cityname','adname','address','name','大类','中类','小类']

anchor_results = {}
distance_stats = {}

for year in range(2013, 2025):
    # Check data availability
    if year + 1 not in GS_data_set or year not in GS_data_set:
        continue
    if year >= 2015 and (year + 1 not in CS_data_set or year not in CS_data_set):
        continue
    
    # Merge GS and CS data for current year
    df1_gs = GS_data_set[year]
    df2_gs = GS_data_set[year + 1]
    
    if year >= 2015:
        df1_cs = CS_data_set[year]
        df2_cs = CS_data_set[year + 1]
        df1 = pd.concat([df1_gs, df1_cs], ignore_index=True)
        df2 = pd.concat([df2_gs, df2_cs], ignore_index=True)
    else:
        df1 = df1_gs
        df2 = df2_gs
    
    # Address filter
    df1 = df1[df1['address'].str.strip().ne('') & df1['address'].str.strip().ne('[]')]
    df2 = df2[df2['address'].str.strip().ne('') & df2['address'].str.strip().ne('[]')]
    
    # Merge to find anchor stations
    merged = pd.merge(df1, df2, on=match_columns, suffixes=('_prev', '_next'))
    anchor_results[year] = merged
    
    # City stats
    city_counts = merged['cityname'].value_counts().to_dict()
    print(f"Year {year}: matched anchor stations per city:", city_counts)
    
    # Distance calculation
    merged['shift_x'] = (merged['wgs84_x_next'] - merged['wgs84_x_prev']) * 100000
    merged['shift_y'] = (merged['wgs84_y_next'] - merged['wgs84_y_prev']) * 100000
    distance_stats[year] = {
        'shift_x': merged['shift_x'].describe().to_dict(),
        'shift_y': merged['shift_y'].describe().to_dict()
    }

Year 2013: matched anchor stations per city: {'上海市': 217, '北京市': 114, '成都市': 61, '济南市': 30, '杭州市': 25, '广州市': 24, '宁波市': 23, '天津市': 23, '深圳市': 20, '济宁市': 18, '哈尔滨市': 16, '长春市': 14, '乌鲁木齐市': 14, '武汉市': 14, '包头市': 12, '南宁市': 12, '青岛市': 10, '南昌市': 10, '运城市': 10, '石家庄市': 9, '沈阳市': 9, '重庆市': 9, '南京市': 9, '东营市': 9, '和田地区': 8, '大连市': 8, '辽阳市': 8, '扬州市': 7, '铜陵市': 7, '衢州市': 7, '日照市': 7, '威海市': 6, '太原市': 6, '南通市': 6, '兰州市': 6, '芜湖市': 6, '绵阳市': 6, '烟台市': 6, '安庆市': 6, '洛阳市': 5, '莱芜市': 5, '中山市': 5, '临沂市': 5, '东莞市': 5, '长沙市': 5, '宿州市': 5, '合肥市': 5, '岳阳市': 5, '苏州市': 5, '海西蒙古族藏族自治州': 5, '珠海市': 4, '无锡市': 4, '吐鲁番市': 4, '西双版纳傣族自治州': 4, '吉林市': 4, '嘉兴市': 4, '厦门市': 4, '三亚市': 4, '恩施土家族苗族自治州': 4, '银川市': 4, '湘潭市': 3, '白银市': 3, '果洛藏族自治州': 3, '连云港市': 3, '鄂州市': 3, '邢台市': 3, '十堰市': 3, '海北藏族自治州': 3, '齐齐哈尔市': 3, '唐山市': 3, '海口市': 3, '盐城市': 3, '惠州市': 3, '呼和浩特市': 3, '昆明市': 3, '巴音郭楞蒙古自治州': 3, '玉溪市': 3, '宣城市': 3, '菏泽市': 3, '淄博市': 3, '德阳市': 3, '绍兴市': 2, '保定市': 2, '郴州市': 2, '新乡市': 2, '绥化市': 2, '湖州市': 2, '衡水市': 2, '德州市': 2

In [4]:
anchor_results[2013]

,name,address,wgs84_x_prev,wgs84_y_prev,tel_prev,pname,cityname,adname,大类,中类,...,paused_prev,wgs84_x_next,wgs84_y_next,tel_next,pname_EN_next,cityname_EN_next,corporation_next,paused_next,shift_x,shift_y
0,九环汽车液化加气站,曹家堰路３８,121.429478,31.215804,,上海市,上海市,长宁区,汽车服务,加气站,...,0,121.434101,31.213939,,Shanghai,Shanghai,Other,0,462.282025,-186.536230
1,加油站（可乐东路）,哈密路８００－１,121.363170,31.205570,,上海市,上海市,长宁区,汽车服务,加油站,...,0,121.367791,31.203698,,Shanghai,Shanghai,CPCC,0,462.106128,-187.194807
2,加油站（双泾寺角落综合小区东）,北翟路１４４２,121.338541,31.226073,,上海市,上海市,长宁区,汽车服务,加油站,...,0,121.343117,31.224171,,Shanghai,Shanghai,CPCC,0,457.611705,-190.177853
3,加油站（天山路）,天山路９０,121.375725,31.217936,,上海市,上海市,长宁区,汽车服务,加油站,...,0,121.380364,31.216084,,Shanghai,Shanghai,CPCC,0,463.850289,-185.203959
4,华江石油加油站（赵重公路）,重固镇,121.181493,31.192347,,上海市,上海市,青浦区,汽车服务,加油站,...,0,121.186178,31.190486,,Shanghai,Shanghai,Other,0,468.507624,-186.088196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,加油站（富兴北路）,新兴东路２,126.472774,45.537524,,黑龙江省,哈尔滨市,双城区,汽车服务,加油站,...,0,126.479009,45.539720,,Heilongjiang Province,Harbin City,CNPC,0,623.466672,219.636454
1070,北街加油站,北大街４３４,126.599548,45.993272,0451-82302588,黑龙江省,哈尔滨市,呼兰区,汽车服务,加油站,...,0,126.605469,45.995045,0451-82302588,Heilongjiang Province,Harbin City,CPCC,0,592.142859,177.296706
1071,泰运加油站,建设路７４０,123.418733,46.417812,,黑龙江省,齐齐哈尔市,泰来县,汽车服务,加油站,...,0,123.425604,46.419834,0452-8226770,Heilongjiang Province,Qiqihar City,Other,0,687.067739,202.196627
1072,加油站（青云小区东）,南马路３０９,123.946617,47.331601,,黑龙江省,齐齐哈尔市,龙沙区,汽车服务,加油站,...,0,123.953506,47.333725,,Heilongjiang Province,Qiqihar City,CNPC,0,688.926481,212.359863


In [5]:
tmp_year = 2016
tmp_df = anchor_results[tmp_year]
result = (
    tmp_df.groupby(['pname_EN_prev','cityname_EN_prev'])
    .agg(
        shift_x_median = ('shift_x', 'median'),
        shift_y_median = ('shift_y', 'median'),
        count = ('shift_x', 'size')
    )
    .reset_index()
    .sort_values(['pname_EN_prev','cityname_EN_prev'])
)
print(tmp_year)
result

2016


,pname_EN_prev,cityname_EN_prev,shift_x_median,shift_y_median,count
0,Anhui Province,Anqing City,-512.254558,239.293436,100
1,Anhui Province,Bengbu City,-566.408916,174.078489,71
2,Anhui Province,Bozhou City,-536.679558,158.782105,101
3,Anhui Province,Chizhou City,-528.350660,250.750639,35
4,Anhui Province,Chuzhou City,-542.778358,199.610377,91
...,...,...,...,...,...
352,hubei province,Xianning City,-544.541948,265.479664,94
353,hubei province,Xiantao City,-601.546268,238.871488,34
354,hubei province,Xiaogan City,-570.431584,233.287057,96
355,hubei province,Yichang City,-571.886579,243.403473,138


In [ ]:
tmp_year = 2024
tmp_df = anchor_results[tmp_year]
result = (
    tmp_df.groupby(['pname_EN_prev','cityname_EN_prev'])
    .agg(
        shift_x_median = ('shift_x', 'median'),
        shift_y_median = ('shift_y', 'median'),
        count = ('shift_x', 'size')
    )
    .reset_index()
    .sort_values(['pname_EN_prev','cityname_EN_prev'])
)
print(tmp_year)
result

2024


,pname_EN_prev,cityname_EN_prev,shift_x_median,shift_y_median,count
0,Anhui Province,Anqing City,0.492665,0.283389,460
1,Anhui Province,Bengbu City,0.358640,0.307338,286
2,Anhui Province,Bozhou City,-0.636643,-0.440757,476
3,Anhui Province,Chizhou City,-0.625592,-0.658118,285
4,Anhui Province,Chuzhou City,0.463895,0.537095,556
...,...,...,...,...,...
363,hubei province,Xianning City,0.249650,0.458814,386
364,hubei province,Xiantao City,-0.602140,-0.607218,118
365,hubei province,Xiaogan City,-0.256093,-0.031540,440
366,hubei province,Yichang City,0.479762,0.146034,767


: 

In [ ]:
import matplotlib.pyplot as plt

def plot_shift_by_province(year):
    """Plot shift_x and shift_y distribution for each province"""
    df = anchor_results[year]
    
    # Aggregate by province and city
    agg_df = (
        df.groupby(['pname_EN_prev', 'cityname_EN_prev'])
        .agg(
            shift_x_median=('shift_x', 'median'),
            shift_y_median=('shift_y', 'median'),
            count=('shift_x', 'size')
        )
        .reset_index()
    )
    
    # Plot for each province
    provinces = agg_df['pname_EN_prev'].unique()
    
    for pname in provinces:
        prov_data = agg_df[agg_df['pname_EN_prev'] == pname]
        
        plt.figure(figsize=(8, 8))
        plt.scatter(prov_data['shift_x_median'], prov_data['shift_y_median'], 
                   s=100, alpha=0.6)
        plt.xlabel('shift_x (meter)')
        plt.ylabel('shift_y (meter)')
        plt.title(f'{pname} - Coordinate Shift Distribution ({year})')
        plt.grid(True, alpha=0.3)
        plt.axhline(y=0, color='k', linestyle='--', linewidth=0.5)
        plt.axvline(x=0, color='k', linestyle='--', linewidth=0.5)
        plt.tight_layout()
        plt.show()
        
        print(f"\n{pname} cities:")
        print(prov_data[['cityname_EN_prev', 'shift_x_median', 'shift_y_median', 'count']])

# Usage
plot_shift_by_province(2016)